In [1]:
from survey_enhance.experiment.initialisation import (
    dataset,
    calibration_parameters,
    Loss,
    create_frs_dataset,
    household_weights,
)

loss = Loss(dataset, calibration_parameters)

initial_loss = loss(household_weights, dataset)

# We're going to use gradient descent to find the household weights that minimise the loss function

# We'll use the Adam optimiser, which is a variant of gradient descent that's more robust to local minima

import torch
from torch import optim

optimiser = optim.Adam([household_weights], lr=0.1)

for i in range(100):
    optimiser.zero_grad()
    loss_value = loss(household_weights, dataset)
    loss_value.backward()
    optimiser.step()
    print(f"Loss: {loss_value.item()}")

TypeError: new() received an invalid combination of arguments - got (Series, requires_grad=bool), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [ ]:
loss_value.requires_grad

False